In [103]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
import pathlib
import tensorflow as tf
import tensorflow_datasets as tfds
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


train_data_path = '/mnt/c/Users/maxxt/Downloads/Projects/Poker Reader/archive/train'

train_data_dir = pathlib.Path(train_data_path).with_suffix('')
test_data_dir = pathlib.Path('/mnt/c/Users/maxxt/Downloads/Projects/Poker Reader/archive/test')

train_data_df = pd.read_csv('/mnt/c/Users/maxxt/Downloads/Projects/Poker Reader/archive/Training_set.csv')

In [104]:
batch_size = 32
image_height = 224
image_width = 224

class_names = list(set(train_data_df['label']))

In [105]:
image_ds = tf.data.Dataset.list_files(str(train_data_dir/'*'), shuffle=False)

In [106]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

image_ds = image_ds.map(lambda x: normalization_layer(read_image(x)))

In [113]:
label_indexes = [class_names.index(card_type) for card_type in train_data_df['label'].values]

label_ds = tf.data.Dataset.from_tensor_slices(label_indexes)

In [114]:
train_ds = tf.data.Dataset.zip((image_ds, label_ds))

In [115]:
train_ds.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [116]:
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  45


2023-04-30 21:50:22.284033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int32 and shape [4776]
	 [[{{node Placeholder/_3}}]]


In [120]:
num_classes = 53

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [121]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [122]:
model.fit(
  train_ds,
  epochs=3
)

Epoch 1/3


ValueError: in user code:

    File "/home/maxxt/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/maxxt/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/maxxt/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/maxxt/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/maxxt/anaconda3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/maxxt/anaconda3/lib/python3.10/site-packages/keras/engine/input_spec.py", line 253, in assert_input_compatibility
   